In [4]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from sklearn.metrics import r2_score
pio.templates.default = 'plotly_white'
plot_template = dict(
    layout=go.Layout({
        'font_size': 8,
        'xaxis_title_font_size': 8,
        'yaxis_title_font_size': 8,
        }   
))
hydro_stations = ['Tangnaihai','Guide','Xunhua']
hydrostation_abbrs = {'Tangnaihai':'TNH','Guide':'GD','Xunhua':'XH'}
hydrostation_channel = {'Tangnaihai':'3','Guide':'14','Xunhua':'9'}

Corect bias of forecats of SWAT+ using cloimate pattern data as input. The scenarios SSP126,SSP245,SSP585 of ACCESS-ESM1-5 was used. The correction method can be found in the following research.
Machine-learning- and deep-learning-based streamflow prediction  in a hilly catchment for future scenarios using CMIP6 GCM data.
The correction equation is as follows:

$Q_{bc} = \overline{Q_{ob}} + \sigma_{ob}/\sigma_{p}(Q_{pred}-\overline{Q_{p}})$

where $Q_{bc}$ is the corrected streamflow, $\overline{Q_{ob}}$ is the historical monthly average streamflow, $\sigma_{ob}$ is the standard deviation of historical monthly streamflow, $Q_{pred}$ is the predicted streamflow, and $\overline{Q_{p}}$ is the historical monthly average predicted streamflow.


In [12]:
for hydro_station in hydro_stations:
    his_obs_df = pd.read_csv(f'../data/{hydro_station.lower()}_natural_monthly_flow.csv',index_col=['date'],parse_dates=['date'])
    his_obs_df = his_obs_df.loc['1983-01-01':'2007-12-31',:]
    his_obs_mean = his_obs_df['flow(m^3/s)'].mean()
    his_obs_std = his_obs_df['flow(m^3/s)'].std()

    his_sim_df = pd.read_csv(f'../result/HistoricalSimulationWithClimatePatternIn(ACCESS)/Channel_{hydrostation_channel[hydro_station]}_Monthly_River-Flow_{hydro_station.lower()}_1983_2007.csv',index_col=['Date'],parse_dates=['Date'])
    his_sim_mean = his_sim_df['Value'].mean()
    his_sim_std = his_sim_df['Value'].std()

    for scenario in ['ssp126','ssp245','ssp585']:
        pred_df_swatp_ssp = pd.read_csv(f'../result/SWATPlusPredUsingClimatePatternData/{hydro_station}_SWATPlus_pred_obs_{scenario}_2015_2019.csv',index_col=['date'],parse_dates=['date'])
        corrected_pred_df_swatp_ssp = his_obs_mean + his_obs_std/his_sim_std*(pred_df_swatp_ssp['pred']-his_sim_mean)
        print('his_obs_std={},his_sim_std={}'.format(his_obs_std,his_sim_std))
        print('pred_df_swatp_ssp={},his_sim_mean={}'.format(pred_df_swatp_ssp['pred'],his_sim_mean))
        
        df = pd.DataFrame({'obs':pred_df_swatp_ssp['obs'].values,'pred':pred_df_swatp_ssp['pred'].values,'corrected_pred':corrected_pred_df_swatp_ssp.values},index=pred_df_swatp_ssp.index)
        df.to_csv(f'../result/SWATPlusPredUsingClimatePatternData/{hydro_station}_SWATPlus_pred_obs_{scenario}_2015_2019_corrected_using_historical_mean_std.csv')
        pred_r2 = r2_score(df['obs'],df['pred'])
        corrected_pred_r2 = r2_score(df['obs'],df['corrected_pred'])
        
        fig = go.Figure()
        fig.add_trace(go.Scatter(x=df.index, y=df['obs'], mode='lines', name='Observed'))
        fig.add_trace(go.Scatter(x=df.index, y=df['pred'], mode='lines', name='Predicted'))
        fig.add_trace(go.Scatter(x=df.index, y=df['corrected_pred'], mode='lines', name='Corrected Predicted'))
        
        
        fig.update_layout(
            title=f'{hydro_station} - {scenario.upper()} Scenario<br>R² (Predicted): {pred_r2:.4f}, R² (Corrected): {corrected_pred_r2:.4f}',
            xaxis_title='Date',
            yaxis_title='Flow (m³/s)',
            legend_title='Data Type',
            template=plot_template
        )
        
        fig.show()

his_obs_std=485.37899185462004,his_sim_std=243.08997947064813
pred_df_swatp_ssp=date
2015-01-01     396.1
2015-02-01     339.4
2015-03-01     288.8
2015-04-01     255.0
2015-05-01     364.3
2015-06-01    1021.0
2015-07-01    1344.0
2015-08-01     875.7
2015-09-01    1362.0
2015-10-01    1058.0
2015-11-01     548.5
2015-12-01     456.6
2016-01-01     390.2
2016-02-01     331.7
2016-03-01     280.5
2016-04-01     268.0
2016-05-01     201.0
2016-06-01     279.4
2016-07-01     547.7
2016-08-01     411.7
2016-09-01     484.5
2016-10-01     360.8
2016-11-01     205.1
2016-12-01     160.3
2017-01-01     177.0
2017-02-01     150.9
2017-03-01     124.4
2017-04-01     131.4
2017-05-01     208.4
2017-06-01     374.3
2017-07-01     531.5
2017-08-01     168.6
2017-09-01     273.6
2017-10-01     511.7
2017-11-01     180.3
2017-12-01     144.4
2018-01-01     277.9
2018-02-01     309.3
2018-03-01     264.8
2018-04-01     235.9
2018-05-01     193.0
2018-06-01     675.6
2018-07-01     822.8
2018-08-01  

his_obs_std=485.37899185462004,his_sim_std=243.08997947064813
pred_df_swatp_ssp=date
2015-01-01     398.2
2015-02-01     341.0
2015-03-01     290.2
2015-04-01     248.9
2015-05-01     360.2
2015-06-01     876.2
2015-07-01     996.2
2015-08-01     830.3
2015-09-01    1032.0
2015-10-01     721.8
2015-11-01     485.0
2015-12-01     414.6
2016-01-01     390.6
2016-02-01     332.5
2016-03-01     278.3
2016-04-01     268.0
2016-05-01     200.7
2016-06-01     282.7
2016-07-01     546.7
2016-08-01     412.8
2016-09-01     484.9
2016-10-01     360.3
2016-11-01     205.4
2016-12-01     160.3
2017-01-01     175.7
2017-02-01     150.5
2017-03-01     123.6
2017-04-01     131.3
2017-05-01     208.1
2017-06-01     374.8
2017-07-01     531.9
2017-08-01     167.2
2017-09-01     273.9
2017-10-01     511.2
2017-11-01     180.0
2017-12-01     145.1
2018-01-01     364.3
2018-02-01     296.3
2018-03-01     207.5
2018-04-01     218.1
2018-05-01     192.9
2018-06-01     676.6
2018-07-01     821.1
2018-08-01  

his_obs_std=485.37899185462004,his_sim_std=243.08997947064813
pred_df_swatp_ssp=date
2015-01-01     397.8
2015-02-01     342.3
2015-03-01     290.6
2015-04-01     249.2
2015-05-01     360.3
2015-06-01     875.4
2015-07-01     995.4
2015-08-01     828.4
2015-09-01    1031.0
2015-10-01     722.0
2015-11-01     485.4
2015-12-01     414.0
2016-01-01     390.7
2016-02-01     332.6
2016-03-01     279.8
2016-04-01     268.6
2016-05-01     201.4
2016-06-01     281.5
2016-07-01     547.0
2016-08-01     414.9
2016-09-01     485.1
2016-10-01     361.5
2016-11-01     205.7
2016-12-01     160.9
2017-01-01     176.9
2017-02-01     150.6
2017-03-01     124.2
2017-04-01     129.8
2017-05-01     208.2
2017-06-01     375.9
2017-07-01     533.4
2017-08-01     168.7
2017-09-01     271.9
2017-10-01     511.7
2017-11-01     180.0
2017-12-01     145.1
2018-01-01     277.2
2018-02-01     353.4
2018-03-01     239.7
2018-04-01     204.3
2018-05-01     333.2
2018-06-01     829.2
2018-07-01     822.0
2018-08-01  

his_obs_std=478.8977425007305,his_sim_std=167.4447528403766
pred_df_swatp_ssp=date
2015-01-01    372.7
2015-02-01    325.2
2015-03-01    280.4
2015-04-01    252.4
2015-05-01    365.8
2015-06-01    701.8
2015-07-01    769.9
2015-08-01    667.0
2015-09-01    680.4
2015-10-01    480.3
2015-11-01    359.7
2015-12-01    316.1
2016-01-01    304.4
2016-02-01    263.6
2016-03-01    224.4
2016-04-01    191.1
2016-05-01    268.4
2016-06-01    364.1
2016-07-01    401.8
2016-08-01    442.5
2016-09-01    426.3
2016-10-01    383.0
2016-11-01    244.9
2016-12-01    205.1
2017-01-01    224.7
2017-02-01    195.2
2017-03-01    163.3
2017-04-01    132.9
2017-05-01    231.5
2017-06-01    347.1
2017-07-01    395.1
2017-08-01    341.5
2017-09-01    458.3
2017-10-01    544.8
2017-11-01    265.1
2017-12-01    223.4
2018-01-01    298.9
2018-02-01    353.2
2018-03-01    308.0
2018-04-01    241.8
2018-05-01    241.6
2018-06-01    485.5
2018-07-01    785.0
2018-08-01    429.9
2018-09-01    503.0
2018-10-01    424

his_obs_std=478.8977425007305,his_sim_std=167.4447528403766
pred_df_swatp_ssp=date
2015-01-01    371.8
2015-02-01    327.3
2015-03-01    278.3
2015-04-01    250.0
2015-05-01    373.8
2015-06-01    710.2
2015-07-01    769.7
2015-08-01    667.7
2015-09-01    680.3
2015-10-01    481.1
2015-11-01    360.0
2015-12-01    317.0
2016-01-01    304.8
2016-02-01    264.3
2016-03-01    222.6
2016-04-01    191.1
2016-05-01    268.2
2016-06-01    366.6
2016-07-01    401.2
2016-08-01    443.5
2016-09-01    426.7
2016-10-01    382.6
2016-11-01    245.2
2016-12-01    205.0
2017-01-01    223.8
2017-02-01    194.8
2017-03-01    162.6
2017-04-01    132.9
2017-05-01    231.3
2017-06-01    347.7
2017-07-01    395.6
2017-08-01    340.5
2017-09-01    458.7
2017-10-01    544.6
2017-11-01    265.1
2017-12-01    224.1
2018-01-01    409.3
2018-02-01    337.9
2018-03-01    239.7
2018-04-01    225.1
2018-05-01    242.1
2018-06-01    486.2
2018-07-01    783.5
2018-08-01    430.0
2018-09-01    503.5
2018-10-01    424

his_obs_std=478.8977425007305,his_sim_std=167.4447528403766
pred_df_swatp_ssp=date
2015-01-01    373.0
2015-02-01    326.9
2015-03-01    280.8
2015-04-01    249.6
2015-05-01    355.8
2015-06-01    692.6
2015-07-01    769.1
2015-08-01    666.2
2015-09-01    679.5
2015-10-01    481.3
2015-11-01    360.3
2015-12-01    316.5
2016-01-01    304.8
2016-02-01    264.3
2016-03-01    223.8
2016-04-01    191.6
2016-05-01    268.8
2016-06-01    365.6
2016-07-01    401.1
2016-08-01    445.1
2016-09-01    426.9
2016-10-01    383.8
2016-11-01    245.6
2016-12-01    205.8
2017-01-01    224.8
2017-02-01    195.0
2017-03-01    163.1
2017-04-01    131.8
2017-05-01    231.1
2017-06-01    348.3
2017-07-01    396.4
2017-08-01    341.5
2017-09-01    456.6
2017-10-01    544.6
2017-11-01    264.9
2017-12-01    224.0
2018-01-01    298.3
2018-02-01    412.7
2018-03-01    273.0
2018-04-01    212.6
2018-05-01    370.9
2018-06-01    723.9
2018-07-01    898.3
2018-08-01    433.4
2018-09-01    505.2
2018-10-01    427

his_obs_std=492.50068172869936,his_sim_std=165.52979012947395
pred_df_swatp_ssp=date
2015-01-01    350.0
2015-02-01    304.4
2015-03-01    258.9
2015-04-01    221.4
2015-05-01    330.2
2015-06-01    676.3
2015-07-01    736.0
2015-08-01    662.7
2015-09-01    681.5
2015-10-01    490.3
2015-11-01    347.3
2015-12-01    306.3
2016-01-01    286.9
2016-02-01    247.2
2016-03-01    208.5
2016-04-01    173.2
2016-05-01    289.3
2016-06-01    352.8
2016-07-01    363.0
2016-08-01    416.8
2016-09-01    417.2
2016-10-01    356.9
2016-11-01    213.5
2016-12-01    180.4
2017-01-01    207.9
2017-02-01    179.5
2017-03-01    147.7
2017-04-01    117.0
2017-05-01    260.2
2017-06-01    366.3
2017-07-01    493.3
2017-08-01    364.1
2017-09-01    444.8
2017-10-01    537.3
2017-11-01    246.6
2017-12-01    209.3
2018-01-01    270.0
2018-02-01    329.6
2018-03-01    287.2
2018-04-01    217.5
2018-05-01    214.5
2018-06-01    446.4
2018-07-01    683.3
2018-08-01    481.3
2018-09-01    505.9
2018-10-01    4

his_obs_std=492.50068172869936,his_sim_std=165.52979012947395
pred_df_swatp_ssp=date
2015-01-01    349.8
2015-02-01    303.3
2015-03-01    258.1
2015-04-01    221.7
2015-05-01    329.7
2015-06-01    677.8
2015-07-01    735.9
2015-08-01    663.6
2015-09-01    681.2
2015-10-01    491.1
2015-11-01    347.7
2015-12-01    307.4
2016-01-01    287.3
2016-02-01    247.9
2016-03-01    206.4
2016-04-01    173.0
2016-05-01    289.0
2016-06-01    355.7
2016-07-01    362.1
2016-08-01    417.8
2016-09-01    417.5
2016-10-01    356.4
2016-11-01    213.8
2016-12-01    180.3
2017-01-01    206.9
2017-02-01    179.0
2017-03-01    147.0
2017-04-01    116.9
2017-05-01    260.0
2017-06-01    366.9
2017-07-01    494.0
2017-08-01    362.9
2017-09-01    445.1
2017-10-01    537.1
2017-11-01    246.4
2017-12-01    210.0
2018-01-01    390.6
2018-02-01    311.3
2018-03-01    213.3
2018-04-01    200.2
2018-05-01    215.6
2018-06-01    447.4
2018-07-01    681.5
2018-08-01    481.1
2018-09-01    506.1
2018-10-01    4

his_obs_std=492.50068172869936,his_sim_std=165.52979012947395
pred_df_swatp_ssp=date
2015-01-01    349.4
2015-02-01    304.6
2015-03-01    258.5
2015-04-01    222.1
2015-05-01    329.9
2015-06-01    677.1
2015-07-01    735.2
2015-08-01    661.7
2015-09-01    680.5
2015-10-01    491.2
2015-11-01    348.0
2015-12-01    306.7
2016-01-01    287.4
2016-02-01    248.0
2016-03-01    207.9
2016-04-01    173.6
2016-05-01    289.6
2016-06-01    354.7
2016-07-01    362.4
2016-08-01    419.7
2016-09-01    417.7
2016-10-01    357.5
2016-11-01    214.0
2016-12-01    180.9
2017-01-01    208.0
2017-02-01    179.2
2017-03-01    147.5
2017-04-01    115.5
2017-05-01    259.8
2017-06-01    367.8
2017-07-01    495.3
2017-08-01    364.4
2017-09-01    443.1
2017-10-01    537.4
2017-11-01    246.2
2017-12-01    210.0
2018-01-01    269.0
2018-02-01    393.1
2018-03-01    249.8
2018-04-01    189.2
2018-05-01    333.3
2018-06-01    632.2
2018-07-01    924.6
2018-08-01    483.0
2018-09-01    506.7
2018-10-01    4

: 